Image colorization project :
We train a CNN to take in greyscale images of ... and output their colorful and plausible colorization.

Imports required :

torch

skimage ? only used for rgb - lab parsing. Maybe we can do that ourselves to make some differentiation with the original project?

numpy

matplotlib

PIL ? only used to open images to rgb. We can use another library

In [65]:
import matplotlib.pyplot as plt 
import numpy as np
from PIL import Image
from skimage import color

import torch
import torch.nn as nn
import torch.utils.data as data
import torch.nn.functional as F

Load data, augment it, transform it to LAB compute stats on colors

In [66]:
#Data loading 

In [67]:
#LAB transform

In [68]:
#Color stats

# need dataset express like that: list of images (as tensor) with dim H x W x 2 ...
images = []

def getDiscretisedColor(a,b,gridSize):
    a = np.round(a/gridSize) * gridSize
    b = np.round(b/gridSize) * gridSize
    return (a,b)

def getMatrixIndex(a,b,gridSize):
    i = (a + 500) / gridSize
    j = (a + 200) / gridSize
    return (i,j)

def getColorValue(i,j,gridSize):
    a = i * gridSize - 500
    b = j * gridSize - 200
    return (a,b)

# Initiate the proba distribution of ab pairs in the images dataset (discretised).
gridSize = 10
colorProbabilities = np.zeros((1000 * gridSize, 400 * gridSize))

# Compute the proba distribution of the ab pairs in the images dataset (discretised).
nbOfAnalysedPixels = 0
for image in images:
    for h in image.shape[0]:
        for w in image.shape[1]:
            (a,b) = getDiscretisedColor(image[h][w][0],image[h][w][1],gridSize)
            (i,j) = getMatrixIndex(a,b,gridSize)
            colorProbabilities[i][j] += 1
            nbOfAnalysedPixels += 1
colorProbabilities / nbOfAnalysedPixels

# + display distribution in 2d plot ? like in paper 

# Get the vector of proba of ab pairs that are "in gamut"
inGamutColors = []
inGamutColorsProbas = []
for i in range (colorProbabilities.shape[0]):
    for j in range (colorProbabilities.shape[1]):
        currentColorPorba = colorProbabilities[i][j]
        if currentColorPorba != 0:
            (a,b) = getColorValue(i,j,gridSize)
            inGamutColors.append((a,b))
            inGamutColorsProbas = currentColorPorba
        
Q = len(inGamutColors) 
p = torch.tensor(inGamutColorsProbas)
print(Q)
print(inGamutColors)
print(inGamutColorsProbas)
Q = 1 # to remove


/tmp/ipykernel_1550923/1663647833.py:34: RuntimeWarning: invalid value encountered in true_divide
  colorProbabilities / nbOfAnalysedPixels


0
[]
[]


In [69]:
#Define pixel weight vector (class rebalancing)

# Set the parameters (from paper, need empirical value).
lambda_uniform = 1/2 
sigma = 5 # gaussian kernel parameter

# Compute a smooth version of the empirical pixel color distribution.
p_smooth = p # how to do that ? gaussian kernel ? to do

# Compute the weight vector.
pixelsWeights = torch.reciprocal((1 - lambda_uniform) * p_smooth + lambda_uniform / Q)

# Normalise the weight vector according to p_smooth (E[W] = 1).
E_W = torch.sum(p_smooth * pixelsWeights)
scale_factor = 1 / E_W
pixelsWeights = scale_factor * pixelsWeights 

ZeroDivisionError: float division by zero

Define and instantiate Convolutional NN

In [ ]:
#NN def

Define optimizer

In [ ]:
#ADAM

Define Loss

In [ ]:
#Convert a true pixel Y to a distribution Z (soft encoding)

# to implement...
def getColorDistribution(Y):
    return 

In [ ]:
#Convert a distribution Z to a true pixel Y (point estimate)

# to implement...
def getColorEstimate(Z):
    return 

In [ ]:
#v(Z_hw) weight in paper (section 2)

def getPixelsWeights(Z):
    W = torch.argmax(Z, dim=2)
    for i in range(W.size(dim=0)):
        for j in range(W.size(dim=1)):
            W[i,j] = pixelsWeights(W[i,j])

    return W


In [ ]:
#loss function 

def multinomialCrossEntropyLoss(Z_estimate, Z):
    W = getPixelsWeights(Z)
    L = - sum(W * torch.sum(Z * torch.log10(Z_estimate), dim=2))

    return L
    


Main training loop and additional function TBD